<a href="https://colab.research.google.com/github/ava-28/Transformer/blob/main/transformer01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GPUtil

In [ ]:
!git clone https://github.com/ava-28/TransformerNo1.git


Cloning into 'TransformerNo1'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git clone https://github.com/YOUR_USERNAME/YOUR_REPO.git



Cloning into 'YOUR_REPO'...
fatal: could not read Username for 'https://github.com': No such device or address


### Scaled Dot-Product Attention

This is the core of the attention mechanism. It takes three inputs: Query (Q), Key (K), and Value (V). It computes the dot product of the query with all keys, divides by the square root of the dimension of the keys ($√ d_k$), and then applies a softmax function to get the attention weights. These weights are then multiplied by the values to get the output.

The formula is:
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Uninstall existing torchtext and torchdata first to ensure a clean install
!pip uninstall -y torchtext torchdata
# Then install the specified versions along with others
!pip install -q torchdata==0.7.1 torchtext==0.18.0 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
Found existing installation: torchdata 0.7.1
Uninstalling torchdata-0.7.1:
  Successfully uninstalled torchdata-0.7.1
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^

In [1]:


import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

ModuleNotFoundError: No module named 'torchtext'

In [3]:
import torchtext
print(f"torchtext version: {torchtext.__version__}")

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None


In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [ ]:
def clones(module, N):
  "produce N identical layers"
  return nn.ModuleList([copy.deepcopy(module) for _ in range (N)])

In [ ]:
from torch.nn.modules.normalization import LayerNorm
class Encoder(nn.Module):
  "Core ecnoder is a stack of N layers"
  def __init__(self, layer, N):
    super(Encoder,self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

def forward(self, x, mask):
  "pass the input (and mask) throuhg each layer in turn"
  for layer in self.layers:
    x = layer(x, mask)
  return self.norm(x)

In [ ]:
"defining the LayerNorm class"
class LayerNorm(nn.Module):
  "Construct a layernorm module"
  def __init__ (self, features, eps = 1e-6):
    super(LayerNorm, self). __init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepim =True)
    std = x.std(-1, keepdim = True )
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection (nn.Module):
  """
  A residual connertion followed by a layer norm.
  """
  def __init__ (self,size, dropout):
    super(SublayerConnection, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, sublayer):
    "Apply residual connection to any sublayer with the same size"
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer (nn.Module):
  "Encoder is made up from self-attention and feed forward"
  def __init__(self, size, self_attn, feed_forward, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    seld.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size,dropout), 2)
    self.size = size




  def forward(self, x, mask):
    #So:  self.sublayer[0] → Add & Norm wrapper for self-attention.   self.sublayer[1] → Add & Norm wrapper for FFN#
    #self.self_attn(x, x, x, mask) means: Query = Key = Value = x.      ---> Tokens attend to each other#

    x = self.sublayer[0](
        x,
        lambda x: self,self_attn(x,x,x, mask)
    )
    return self.sublayer [1] (x, self.feed_forward)

Here we start with the Decoder:


the decoder is cpomposed of a stack pf N = 6 identical layers. So we use the same N here as well.





In [ ]:
class Decoder(nn.Module):
  "Generic N layer decoder with masking"

    def __init__(self, layer, N):
      super(Decoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm (layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
      for layer in self.layers:
        x = layer (x, memory, src_mask, tgt_mask)
      return self.norm

we have three parts here:
1. self attention (over the output of the the decoder)
2. sequence attention (over the output of both encoder and the decoder0
3. Feed forward

we have to also employ the residual connections around each of the sub-layers of the decoder, followed by layer normalization.

In [ ]:
class DecoderLayer(nn.Module):
  "Decoder is made of self-attn, and feed froward. we have the size for it, the initial input of the decoder via self attention(what it had produced before)"
  "the multi haed attention for the encoder decoder layer, the feed forwrd, and the dropout"

  def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
    super(DecoderLayer, self).__inti__()
    self.size = size
    self.self_attn = self_attn
    self.src_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 3)


    def forward(self, x, memory, src_mask, tgt_mask):

      m = memory
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
      x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
      return self.sublayer[2](x, self.feed_forward)

In [ ]:
def subsequent_mask (size):
  "Mask out subsequent positions."
  attn_shape = (1, size, size)
  subsequence_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
      torch.uint8
      # torch.uint8 = Casts the tensor to an older boolean-ish integer type (0/1)#
  )
  return subsequent_mask == 0

In [ ]:
RUN_EXAMPLES = True
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None


def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )


show_example(example_mask)

 The code is showing the position each tagget word (row) is allowed to look at - words cant see positions father thn themselves.


 **Attention**
 now here we are going to define the attention structure.


In [ ]:
def attention(query, key, value, mask = None, droput = None)
  "compute Scaled dot product of of Q and K"
  d_k = query.size(-1) "the query is tensor of shape (batch_size, sequence_length, features). then .size(-1) will capture the last dimention - feature"
  scores = torch.matmul(query, key.transpose(-2, -1))/ math.sqrt(d_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0, -1e9) "mask == 0 is a boolean mask where ture is when the position should be ignored"
    ".masked_filled is going to fill any true position with the value -1e9"
  p_attn = scores.softmax(dim = -1) "gets appied to the attention scores of softmax along the last layer.

  if dropout is not None:
    p_attn = dropout(p_attn)

  return troch.matmul(p_attn, value), p_attn

**MULTI HEAD ATTENTION**
concates the heads where the heads are made from the attention layers: Attention (QW,KW,VW)

d_k = d_v = d_model/h = 64

In [ ]:
class MultiHeadedAttention(nn.Module)
def __init__(self, h, d_model, dropout=0.1):
  "Take in the model size and numbers of heads."

  super(MultiHeadedAttention, self).__init__()
  assert d_model % h == 0 "assert is used to double check if: 1. d_model is divisible by the number of heads as each layer of heads will process some part of the data"
  self.d_k = d_model // h "2. the division must give out an integer."
  self.h = h
  self.linears = clones(nn.Linear(d_model, d_model), 4)
  self.attn = None
  self.dropout = nn.Dropout(p = dropout)



def forward(self, query, key, value, mask=None):
  "now we are at feed forward"
  if mask is not None:
    # apply some mask on the layers of h
    mask = mask.unsqueeze (1) #is used to add a new dimension of size 1 to a tensor at a specified position
  nbatches = query.size (0) # query = (batch_size, sequence_length, embedding_dimension)

  # 1) we start from doing all the linear projections in batch from d_model => h
# x d_k
query,key, value = [
    lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1 ,2)
    for lin, x in zip(self.linears, (query, key, value)) #this function pairs up elements
]

# 2) apply attention on all project vectors in the batch.

x,  self.attn = attention(
    query, key, value, mask=mask, dropout=self.dropout

)


# 3) "Concat" using a view and apply a final linear

x = (
    x.transpose(1, 2)
    .contiguous() # is related to how tensors are stored in memory
    .view(nbatches, -1, self.h * self.d_k)
)

del query
del key
del value
return self.linears [-1](x)

Position-wise Feed-Forward Networks

FFN(x) = max(0, xW1 + b1)W2 + b2


In [ ]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
    super(PositionwiseFeedForward, self).__init__()
    self.w_1 = nn.Linear(d_model, d_ff) #the biases for each of these terms are included in the nn.Linear
    self.w_2 = nn.Linear(d_ff, d_model) #the biases for each of these terms are included in the nn.Linear
    self.dropout = nn.Dropout(dropout)

  def forward(self, x)
  return self.w_2(self.dropout(self.w_1(x).relu()))

**EMBEDDINGS AND SOFTMAX**

convert the input tokens and output tokens to vctor dimension d_model

In [ ]:
class Embeddings(nn.Module):
  def __init__(self, d_model, vocab):
    super(Embeddings, self).__init__()
    self.lut = nn.Embedding(vocab, d_model)
    self.d_model = d_model

  def forward(self, x):
    return self.lut(x) * math.sqrt(self.d_model)

**POSITIONAL ENCODING**

usibg the frequenceis, we find the position of each token. in here the position is pos, i is the dimension. each dimension of the positional encoding corresponds to a sinusoid.

In [ ]:
class PositionalEncoding(nn.Module):
  "implementing the PE function."

  def __init__(self, d_model, dropout, max_len=5000) #the max sequenc length the model can handel
  super(PositionalEncoding, self).__init__()
  sel.dropout = nn.Dropout(p=dropout)

  #compute the positional encodings once in log space

  pe = torch.zeros(max_len, d_model) #torch.zeros = tensor filled with the scalar 0
  position = torch.arange(0, max_len).unsqueeze(1) #This function creates a 1D tensor with values from 0 up to (but not including) max_len
  div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

  pe[:, 0::2] = torch.sin(position * div_term)
  pe[:, 1::2] = torch.cos(position * div_term)
  pe = pe.unsqueeze(0)

# Task
Please manually restart the Colab runtime (Runtime -> Restart runtime) to apply the package installation changes. After the runtime has restarted, please approve to continue.

## User Restart Runtime

### Subtask:
Manually restart the Colab runtime to apply the package installation changes and ensure all dependencies are loaded correctly.


#### Instructions
1. Go to the 'Runtime' menu at the top of the Colab notebook.
2. Select 'Restart runtime' from the dropdown menu.
3. Confirm the restart when prompted.

## Execute Modified Cells

### Subtask:
After the runtime restart, execute cell `6FtuxHjR-CLv` and then cell `LPTmtBvuxS7T` to ensure the correct versions of `torchdata` and `torchtext` are installed and `RUN_EXAMPLES` is defined. This step also implicitly handles `torchtext` import, and if it still fails, further investigation might be needed.


**Reasoning**:
The first step in the subtask is to execute cell `6FtuxHjR-CLv` to install necessary packages and ensure correct versions of `torchdata` and `torchtext`.



In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Uninstall existing torchtext and torchdata first to ensure a clean install
!pip uninstall -y torchtext torchdata
# Then install the specified versions along with others
!pip install -q torchdata==0.7.1 torchtext==0.18.0 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

**Reasoning**:
After executing cell `6FtuxHjR-CLv` to install the correct package versions, the next logical step is to execute cell `VhoxD8OBpx-L` which defines necessary imports and global variables like `RUN_EXAMPLES`. This is crucial for `LPTmtBvuxS7T` to execute without a `NameError`.



In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

**Reasoning**:
The OSError when importing `torchtext` is likely due to an ABI incompatibility caused by the `torchdata` version conflict reported during package installation. I will modify the installation command to install `torchdata==0.11.0` as required by `torchtune`, hoping it resolves the compatibility issue for `torchtext`.



In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip uninstall -y torchtext torchdata
!pip install -q torchdata==0.11.0 torchtext==0.18.0 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

## User Restart Runtime

### Subtask:
Manually restart the Colab runtime to apply the package installation changes and ensure all dependencies are loaded correctly.

#### Instructions
1. Go to the 'Runtime' menu at the top of the Colab notebook.
2. Select 'Restart runtime' from the dropdown menu.
3. Confirm the restart when prompted.

**Reasoning**:
The runtime is assumed to have been restarted. According to the updated subtask, the next step is to re-execute cell `6FtuxHjR-CLv` to ensure correct package versions are installed after the restart.



In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Uninstall existing torchtext and torchdata first to ensure a clean install
!pip uninstall -y torchtext torchdata
# Then install the specified versions along with others
!pip install -q torchdata==0.11.0 torchtext==0.18.0 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

**Reasoning**:
After successfully re-executing cell `6FtuxHjR-CLv` (which installs packages) and assuming a runtime restart, the next crucial step is to execute cell `VhoxD8OBpx-L`. This cell defines essential imports, including `torchtext` which previously caused an `OSError`, and also defines the global variable `RUN_EXAMPLES`. Executing this cell is a prerequisite for `LPTmtBvuxS7T` to run without a `NameError` and to check if the `torchtext` import issue is finally resolved.



In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

**Reasoning**:
The persistent `OSError` with `torchtext` despite reinstallations and runtime restarts indicates an ABI incompatibility between `torchtext==0.18.0` and the underlying PyTorch installation in the Colab environment. Downgrading `torchtext` to a slightly older version, `0.17.0`, which is often more compatible with various PyTorch versions, might resolve this symbol loading issue. The `torchdata==0.11.0` will be kept to maintain `torchtune` compatibility.



In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip uninstall -y torchtext torchdata
!pip install -q torchdata==0.11.0 torchtext==0.17.0 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

**Reasoning**:
The previous attempt to install `torchtext==0.17.0` failed as it was not found. Consulting the available versions from the error message, `torchtext==0.17.2` is a viable older version. This version will be installed, keeping `torchdata==0.11.0`, to try and resolve the persistent `OSError` due to ABI incompatibility with the underlying PyTorch installation.



In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip uninstall -y numpy pandas torchtext torchdata
!pip install -q numpy pandas torchdata==0.11.0 torchtext==0.17.2 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm